# Katz Centrality

In this notebook, we will use both cuGraph and NetworkX to compute the Katz centrality of each vertex in our test dataset. NetworkX also contains a Numpy implementation that will used. The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees
* Created:   10/15/2019
* Last Edit: 10/15/2019

RAPIDS Versions: 0.10.0   

Test Hardware

* GV100 32G, CUDA 10.0


## Introduction
Katz centrality is a measure of the relative importance of a vertex within the graph based on measuring the influence across the total number of walks between vertex pairs. 

<img src="https://latex.codecogs.com/gif.latex?C_{katz}(i)&space;=&space;\sum_{k=1}^{\infty}&space;\sum_{j=1}^{n}&space;\alpha&space;^k(A^k)_{ji}" title="C_{katz}(i) = \sum_{k=1}^{\infty} \sum_{j=1}^{n} \alpha ^k(A^k)_{ji}" />

See [Katz on Wikipedia](https://en.wikipedia.org/wiki/Katz_centrality) for more details on the algorithm.

To compute the Katz centrality scores for a graph in cuGraph we use:<br>
__df = cugraph.katz_centralityu(G,alpha=0.1, max_iter=100, tol=1.0e-6, nstart=None, normalized=True)__

    G: cugraph.Graph object
    alpha: float, Attenuation factor. default is 0.1
    max_iter: int, The maximum number of iterations before an answer is returned. 
            This can be used to limit the execution time and do an early exit 
            before the solver reaches the convergence tolerance. If this value 
            is lower or equal to 0 cuGraph will use the default value, which is 100
    tol: float, Set the tolerance the approximation, this parameter should be a small 
            magnitude value. The lower the tolerance the better the approximation. If 
            this value is 0.0f, cuGraph will use the default value which is 0.00001. 
            Setting too small a tolerance can lead to non-convergence due to numerical 
            roundoff. Usually values between 0.01 and 0.00001 are acceptable.
    nstart:cuDataFrame, GPU Dataframe containing the initial guess for katz centrality. 
            Default is None
    normalized:bool, If True normalize the resulting katz centrality values.  
            Default is True

Returns:

    df: a cudf.DataFrame object with two columns:
        df['vertex']: The vertex identifier for the vertex
        df['pagerank']: The pagerank score for the vertex



The value of _alpha_ should be<br>
<img src="https://latex.codecogs.com/gif.latex?\alpha&space;<&space;\frac{1}{\lambda&space;_{max}&space;}" title="\alpha < \frac{1}{\lambda _{max} }" />


### _NOTICE_
There is a difference between how cuGraph and how NetworkX computes the Katz centrality score.  That difference leads to the scores not matching.  cuGraph does not currently support the 'beta' and 'weight' parameters as seen in the corresponding networkX call. The cuGraph implementation is based on a relaxed version of Katz defined by Foster with a reduced computational complexity of O(n+m)

    Foster, K.C., Muth, S.Q., Potterat, J.J. et al.
    Computational & Mathematical Organization Theory (2001) 7: 275.
    https://doi.org/10.1023/A:1013470632383


#### cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [20]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [21]:
# NetworkX libraries
import networkx as nx

### Some Prep

In [22]:
# define the parameters 
max_iter = 100  # The maximum number of iterations
tol = 0.00001   # tolerance

In [23]:
# Define the path to the test data  
datafile='../data/karate-data.csv'

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [24]:
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

### Create a Graph 

In [25]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

In [26]:
# compute degree and get the max
degree = G.degrees()
lamda = degree['out_degree'].max()

In [27]:
print("The max degree is " + str(lamda))

The max degree is 17


### Call the Karz algorithm

In [28]:
alpha = 1 / lamda

In [29]:
# Call cugraph.katz_centrality to get the Katz scores
gdf_katz = cugraph.katz_centrality(G, alpha=alpha)

_It was that easy!_  

----

Let's now look at the results

In [30]:
# Find the most important vertex using the scores
# This methods should only be used for small graph
def find_top_scores(_df) :
    m = _df['katz_centrality'].max()
    return _df.query('katz_centrality >= @m')
        

In [31]:
top_df = find_top_scores(gdf_katz)
top_df

,vertex,katz_centrality
34,34,0.433606


In [32]:
# let's sort the data and look at the top 5 vertices
gdf_katz.sort_values(by='katz_centrality', ascending=False).head(5)

,vertex,katz_centrality
34,34,0.433606
1,1,0.416119
33,33,0.327817
3,3,0.296538
2,2,0.256625


---

## Now compute using NetworkX

In [33]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [34]:
k_nx = nx.katz_centrality(Gnx, alpha=alpha, max_iter=max_iter, tol=tol)

In [35]:
k_nx_s = sorted(((value, key) for (key,value) in k_nx.items()), reverse=True)

In [36]:
k_nx_s[:5]

[(0.27758732063256364, '34'),
 (0.2709998485802582, '1'),
 (0.23773581168671487, '33'),
 (0.22595292741481365, '3'),
 (0.21091768975920322, '2')]

As mentioned, the scores are different but the ranking is the same.

In [37]:
# The Numpy version
k_nx_mp = nx.katz_centrality_numpy(Gnx, alpha=alpha)

In [38]:
sorted(((value, key) for (key,value) in k_nx_mp.items()), reverse=True)[:5]

[(0.2775875628109348, '34'),
 (0.27100006570440377, '1'),
 (0.23773599523859307, '33'),
 (0.22595315380272363, '3'),
 (0.21091783569614825, '2')]

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___